In [1]:
# pip install datasets transformers torch
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.5 MB/s eta 0:00:0000:01


**Libraries**

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

Data = "/kaggle/input/cleaned-pashto-squad/cleaned_SQuAD_Pashto.csv"
data = pd.read_csv(Data)
train_data, test_data = train_test_split(data, test_size=0.15, random_state=40)
train_data.to_csv("/kaggle/working/train_data.csv", index=False)
test_data.to_csv("/kaggle/working/test_data.csv", index=False)
print("Dataset split complete. Train and test datasets saved.")


Dataset split complete. Train and test datasets saved.


In [3]:
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments
import torch
import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer, 
    AutoModelForQuestionAnswering, 
    TrainingArguments, 
    Trainer
)

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import shutil


**Data Loader**

In [4]:
def load_and_split_data(
    input_csv, 
    train_ratio=0.7, 
    test_ratio=0.15, 
    random_seed=42
):
    df = pd.read_csv(input_csv)
    df = df.sample(frac=1, random_state=random_seed).reset_index(drop=True)
    total_samples = len(df)
    train_end = int(total_samples * train_ratio)
    val_end = train_end + int(total_samples * test_ratio)
    
    train_df = df.iloc[:train_end]
    val_df = df.iloc[train_end:val_end]
    test_df = df.iloc[val_end:]
    
    def transform_subset(subset_df):
        records = []
        for _, row in tqdm(subset_df.iterrows(), total=len(subset_df), 
                            desc="Transforming data"):
            context = ' '.join(row['context']) if isinstance(row['context'], list) else row['context']
            
            answer_start = context.find(row['answer']) \
                if not row['is_impossible'] else -1
            
            record = {
                "id": row.get('id', ''),
                "title": row.get('title', ''),
                "context": context,
                "question": row['question'],
                "answer": row['answer'],
                "answer_start": answer_start,
                "is_impossible": row['is_impossible']
            }
            records.append(record)
        return records
    
    train_records = transform_subset(train_df)
    val_records = transform_subset(val_df)
    test_records = transform_subset(test_df)
    
    train_dataset = Dataset.from_pandas(pd.DataFrame(train_records))
    val_dataset = Dataset.from_pandas(pd.DataFrame(val_records))
    test_dataset = Dataset.from_pandas(pd.DataFrame(test_records))
    
    return DatasetDict({
        "train": train_dataset, 
        "validation": val_dataset, 
        "test": test_dataset
    })

**Pre Processing**

In [5]:
def preprocess_function(tokenizer, examples, max_length=384, stride=128):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    
    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answer"]
    answer_starts = examples["answer_start"]
    start_positions = []
    end_positions = []
    
    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer_starts[sample_idx]
        end_char = answer_starts[sample_idx] + len(answer)
        sequence_ids = inputs.sequence_ids(i)
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1
        
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)
            
            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)
    
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    
    return inputs

**Train**

In [6]:
def train_qa_model(
    data, 
    model_name="roberta-base", 
    num_epochs=5,
    learning_rate=2e-5,
    batch_size=32
):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForQuestionAnswering.from_pretrained(model_name)
    processed_data = data.map(
        lambda x: preprocess_function(tokenizer, x), 
        batched=True, 
        remove_columns=data["train"].column_names
    )
    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,  
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        weight_decay=0.01,
        logging_dir="./logs",
        logging_steps=10,
        save_steps=500,
        save_total_limit=2,
    )
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=processed_data["train"],
        eval_dataset=processed_data["validation"],
        tokenizer=tokenizer,
    )
    trainer.train()
    model.save_pretrained("./fine_tuned_model")
    tokenizer.save_pretrained("./fine_tuned_model")
    try:
        shutil.make_archive("./fine_tuned_model_archive", 'zip', "./fine_tuned_model")
        print(f"Model saved and zipped to ./fine_tuned_model_archive.zip")
    except Exception as e:
        print(f"Error creating zip archive: {e}")
    
    return trainer, tokenizer, model


In [7]:
input_csv = "/kaggle/working/train_data.csv"
dataset = load_and_split_data(input_csv)
trainer, tokenizer, model = train_qa_model(dataset)

Transforming data: 100%|██████████| 638/638 [00:00<00:00, 16858.60it/s]


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_23/612374700.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctr

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.340200,0.338157
2,0.203800,0.309232
3,0.275300,0.291818
4,0.203800,0.289530
5,0.165700,0.290670


Model saved and zipped to ./fine_tuned_model_archive.zip


**Inference**

In [36]:
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

def calculate_exact_match(predicted_answer, actual_answer):
    if isinstance(predicted_answer, list):
        predicted_answer = " ".join(predicted_answer)  
    if isinstance(actual_answer, list):
        actual_answer = " ".join(actual_answer)  
    return 1 if predicted_answer.strip().lower() == actual_answer.strip().lower() else 0

def calculate_f1_score(predicted_answer, actual_answer):
    if isinstance(predicted_answer, list):
        predicted_answer = " ".join(predicted_answer)
    if isinstance(actual_answer, list):
        actual_answer = " ".join(actual_answer)
    
    pred_tokens = set(predicted_answer.strip().lower().split())
    actual_tokens = set(actual_answer.strip().lower().split())
    precision = len(pred_tokens & actual_tokens) / len(pred_tokens) if pred_tokens else 0
    recall = len(pred_tokens & actual_tokens) / len(actual_tokens) if actual_tokens else 0
    return 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

def calculate_cosine_similarity(predicted_answer, actual_answer):
    if isinstance(predicted_answer, list):
        predicted_answer = " ".join(predicted_answer)
    if isinstance(actual_answer, list):
        actual_answer = " ".join(actual_answer)
    
    # Remove stop words and clean the text
    def preprocess_text(text):
        # Remove punctuation and convert to lowercase
        text = re.sub(r'[^\w\s]', '', text.lower())
        return text
    
    # Check if the texts are not empty after preprocessing
    processed_pred = preprocess_text(predicted_answer)
    processed_actual = preprocess_text(actual_answer)
    
    if not processed_pred or not processed_actual:
        return 0.0  # Return 0 similarity if either text is empty
    
    vectorizer = CountVectorizer(stop_words='english')
    try:
        vectorizer.fit([processed_pred, processed_actual])
        vectorizer_matrix = vectorizer.transform([processed_pred, processed_actual])
        cos_sim = cosine_similarity(vectorizer_matrix[0:1], vectorizer_matrix[1:2])
        return cos_sim[0][0]
    except ValueError:
        # Fallback if vocabulary creation fails
        return 0.0

def display_results_qa_model(tokenizer, model, dataset, num_examples=5):
    qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)
    examples = dataset["validation"].select(range(num_examples))
    
    total_exact_match = 0
    total_f1_score = 0
    total_cosine_sim = 0
    
    print("Evaluating on all examples...\n")
    
    for i, example in enumerate(examples):
        context = example['context']
        question = example['question']
        actual_answer = example['answer'] 
        prediction = qa_pipeline(question=question, context=context)
        predicted_answer = prediction['answer']
        predicted_answer = predicted_answer.replace("••", "")

        exact_match = calculate_exact_match(predicted_answer, actual_answer)
        f1_score = calculate_f1_score(predicted_answer, actual_answer)
        cosine_sim = calculate_cosine_similarity(predicted_answer, actual_answer)
        
        total_exact_match += exact_match
        total_f1_score += f1_score
        total_cosine_sim += cosine_sim
        
        # print(f"Example {i+1}:")
        # print(f"Context: {context}\n")
        # print(f"Question: {question}")
        # print(f"Predicted Answer: {predicted_answer}")
        # print(f"Actual Answer: {actual_answer}")
        # print(f"Score: {prediction['score']:.4f}")
        # print(f"Exact Match: {exact_match}")
        # print(f"F1 Score: {f1_score:.4f}")
        # print(f"Cosine Similarity: {cosine_sim:.4f}")
        # print("-" * 80)

    avg_exact_match = total_exact_match / num_examples
    avg_f1_score = total_f1_score / num_examples
    avg_cosine_sim = total_cosine_sim / num_examples
    
    print("Overall Evaluation Results:")
    print(f"Average Exact Match: {avg_exact_match:.4f}")
    print(f"Average F1 Score: {avg_f1_score:.4f}")
    print(f"Average Cosine Similarity: {avg_cosine_sim:.4f}")

In [37]:
model_path = "./fine_tuned_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForQuestionAnswering.from_pretrained(model_path)
input_csv2 = "/kaggle/input/cleaned-pashto-squad/cleaned_SQuAD_Pashto.csv"
dataset = load_and_split_data(input_csv2)  
display_results_qa_model(tokenizer, model, dataset, num_examples=750)

Transforming data: 100%|██████████| 750/750 [00:00<00:00, 16186.64it/s]
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Evaluating on all examples...

Overall Evaluation Results:
Average Exact Match: 0.1880
Average F1 Score: 0.3610
Average Cosine Similarity: 0.3878


**Pertubed Inference**

In [ ]:
import pandas as pd
import re

def load_and_clean_dataset(input_csv):
    data = pd.read_csv(input_csv).dropna()
    altered_data = data.iloc[:, [2, 3, 4]]
    def clean_text(text):
        cleaned_text = re.sub(r'[^\u0600-\u06FFa-zA-Z0-9\s,؟۔!"\'()-]', '', text)
        return cleaned_text.strip()

    altered_data.iloc[:, 0] = altered_data.iloc[:, 0].apply(clean_text)
    altered_data.iloc[:, 1] = altered_data.iloc[:, 1].apply(clean_text)
    altered_data.iloc[:, 2] = altered_data.iloc[:, 2].apply(clean_text)

    altered_data.columns = ['altered_context', 'altered_question', 'answer'] 
    return altered_data

# data_path = '/kaggle/input/cleaned-pashto-squad/cleaned_SQuAD_Pashto.csv'
# altered_data = load_and_clean_dataset(data_path)
# print("Cleaned dataset preview:")
# print(altered_data.head())


from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_exact_match(predicted_answer, actual_answer):
    if isinstance(predicted_answer, list):
        predicted_answer = " ".join(predicted_answer)
    if isinstance(actual_answer, list):
        actual_answer = " ".join(actual_answer)
    return 1 if predicted_answer.strip().lower() == actual_answer.strip().lower() else 0

def calculate_f1_score(predicted_answer, actual_answer):
    if isinstance(predicted_answer, list):
        predicted_answer = " ".join(predicted_answer)
    if isinstance(actual_answer, list):
        actual_answer = " ".join(actual_answer)

    pred_tokens = set(predicted_answer.strip().lower().split())
    actual_tokens = set(actual_answer.strip().lower().split())
    precision = len(pred_tokens & actual_tokens) / len(pred_tokens) if pred_tokens else 0
    recall = len(pred_tokens & actual_tokens) / len(actual_tokens) if actual_tokens else 0
    return 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

def calculate_cosine_similarity(predicted_answer, actual_answer):
    if isinstance(predicted_answer, list):
        predicted_answer = " ".join(predicted_answer)
    if isinstance(actual_answer, list):
        actual_answer = " ".join(actual_answer)

    vectorizer = CountVectorizer().fit_transform([predicted_answer, actual_answer])
    cos_sim = cosine_similarity(vectorizer[0:1], vectorizer[1:2])
    return cos_sim[0][0]

def evaluate_qa_perturbed(dataset, model, context_col, question_col, answer_col):
    qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

    altered_results = []
    altered_f1_scores = []

    for index, row in dataset.iterrows():
        context = row[context_col]
        question = row[question_col]
        actual_answer = row[answer_col]
        prediction = qa_pipeline(question=question, context=context)
        predicted_answer = prediction['answer']
        
        exact_match = calculate_exact_match(predicted_answer, actual_answer)
        f1_score = calculate_f1_score(predicted_answer, actual_answer)
        cosine_sim = calculate_cosine_similarity(predicted_answer, actual_answer)

        altered_results.append({
            'context': context,
            'question': question,
            'predicted_answer': predicted_answer,
            'actual_answer': actual_answer,
            'exact_match': exact_match,
            'f1_score': f1_score,
            'cosine_similarity': cosine_sim
        })

        altered_f1_scores.append(f1_score)
    
    return altered_f1_scores, altered_results

model_path = "./fine_tuned_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForQuestionAnswering.from_pretrained(model_path)
altered_data_path = "/kaggle/input/pertubated-pashto/pertubated_pashto_translated.csv"
altered_data = load_and_clean_dataset(altered_data_path)
altered_f1_scores, altered_results = evaluate_qa_perturbed(altered_data, model, 'altered_context', 'altered_question', 'answer')
altered_results_df = pd.DataFrame(altered_results)
altered_results_df.to_csv("altered_results.csv", index=False, encoding='utf-8')

print("Evaluation results saved to 'altered_results.csv'")


Cleaned dataset preview:
            altered_context altered_question  \
0  56d43c5f2ccc5a1400d830aa           بیونسی   
1  56bf6e823aeaaa14008c9629           بیونسی   
2  56d43f7e2ccc5a1400d830cb           بیونسی   
3  56be8a583aeaaa14008c9097           بیونسی   
4  56d45abf2ccc5a1400d830e6           بیونسی   

                                              answer  
0  'بیونس ګیزیل نولس-کارټر (bi39؛jnse bee-YON-say...  
1  'د 2005 کال د جون په میاشت کې د تقدیر ماشوم له...  
2  'د ځان تشریح شوي عصري ښځینه فیمینیسټ، بیونسي س...  
3  'بیونسي په فریډریکسبرګ، ټیکساس کې د سینټ مریم ...  
4  'په اته کلنۍ کې، بیونسي او د ماشومتوب ملګري کی...  


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
